In [1]:
using Revise
using LinearAlgebra
using Tonari
using Plots
using Random
using Meiran
using Distributions
using BenchmarkTools
rng = MersenneTwister(1234);

Define the power spectral densities for the two processes and the cross-spectral density between them 

In [2]:
# p1 = SingleBendingPowerLaw(1.0, 0.30, 4e-2, 3.5)
# p2 = SingleBendingPowerLaw(1.0, 0.30, 4e-2, 3.5)
# Δϕ = ConstantTimeLag(5.4)
# cs = CrossSpectralDensity(p1, p2, Δϕ)

# T = 120.0
# Δt = 1.0
p1 = SingleBendingPowerLaw(1.0, 0.30, 4e-2, 3.5)
p2 = SingleBendingPowerLaw(1.0, 0.550, 8e-2, 2.95)
Δϕ = ConstantTimeLag(5.4)
cs = CrossSpectralDensity(p1, p2, Δϕ)

T = 100.0
Δt = 1.0

simu = Simulation(cs, T, Δt)
t, x, xerr = sample(rng,simu,1,error_size=0.1)
x₁,x₂ = x[1][:],x[2][:]
σ_x₁, σ_x₂ = xerr[1][:], xerr[2][:]

# scatter(t,x₁,yerr=σ_x₁,label="X₁")
# scatter!(t,x₂,yerr=σ_x₂,label="X₂")

([0.0595778389003332, 0.006343410180129825, 0.03642499446488528, 0.0769950090480818, 0.004387484165350737, 0.010324941493221346, 0.008745348328058457, 0.0326914334759345, 0.024613094075391934, 0.0014161312298762675  …  0.01788697142903785, 0.005073070168824896, 0.024325456028034022, 0.02706423306696359, 0.024490945354815434, 0.006435819475766143, 0.011004091407632973, 0.0027098522819354845, 0.002035172328642939, 0.00145986170148586], [0.032333206607358805, 0.09139355387812492, 0.041710651449274834, 0.00490163919189618, 0.03285608234876701, 0.038613033567775415, 0.017817845612490706, 0.01413682407260681, 0.0487689161657269, 0.04461639814295404  …  0.009175733953240107, 0.008098299815197751, 0.01831407919981092, 0.01835405711888537, 0.048026313134803995, 0.10073939922700471, 0.10892009238493333, 0.11665484669154447, 0.03964350292618359, 0.06855726531906445])

In [3]:
# using DelimitedFiles
# writedlm("data.txt", [t x₁ x₂ σ_x₁ σ_x₂])
# mean(x₁), mean(x₂), var(x₁), var(x₂)


In [4]:
S_low, S_high = 5, 5
f0, fM = 1 / T / S_low, 1 / (2Δt) * S_high
J = 100

100

In [5]:
function approximate_cross_spectral_density!(
	ωⱼ::AbstractVector{Float64},
	zⱼ::AbstractVector{Float64},
	a_𝓟₁::AbstractVector{Float64},
	a_𝓟₂::AbstractVector{Float64},
	a_𝓒₁₂::AbstractVector{Float64},
	a_τ::AbstractVector{Float64},
	cs::CrossSpectralDensity,
	f0::Float64,
	fM::Float64,
	J::Int64,
)
	# first basis function centred at 0.
	ωⱼ[1] = 2 * f0#fⱼ[1]
	zⱼ[1] = 0.0

	q = (fM / f0)^(1.0 / (J - 1))
	# remaining basis functions
	for j in 2:J
		fⱼ, fⱼ₋₁ = f0 * q^(j - 1), f0 * q^(j - 2)
		ωⱼ[j] = fⱼ - fⱼ₋₁
		zⱼ[j] = fⱼ₋₁ + ωⱼ[j] / 2
	end

	a_𝓟₁[1] = cs.𝓟₁(f0) 
	a_𝓟₂[1] = cs.𝓟₂(f0)  
	a_𝓒₁₂[1] = √a_𝓟₁[1] * √a_𝓟₂[1]
	a_τ[1] = cs.Δφ(f0)
 

	zv = zⱼ[2:J]
	a_𝓟₁[2:end] = cs.𝓟₁.(zv)
	a_𝓟₂[2:end] = cs.𝓟₂.(zv)
	a_𝓒₁₂[2:end] = @. √a_𝓟₁[2:end] * √a_𝓟₂[2:end]
	a_τ[2:end] = cs.Δφ(zv)
end
# function approximated_covariance2(τ::Float64, aⱼ::AbstractVector{Float64}, ωⱼ::AbstractVector{Float64}, zⱼ::AbstractVector{Float64}, J::Int64)
# 	R = aⱼ[1] * ωⱼ[1] * cos(2π * zⱼ[1] * τ) * sinc(ωⱼ[1] * τ)
# 	for j in 2:J
# 		R += 2aⱼ[j] * ωⱼ[j] * cos(2π * zⱼ[j] * τ) * sinc(ωⱼ[j] * τ)
# 	end
# 	return R
# end
# function approximated_cross_covariance2(τ::Float64, aⱼ::AbstractVector{Float64}, τⱼ::AbstractVector{Float64}, ωⱼ::AbstractVector{Float64}, zⱼ::AbstractVector{Float64}, J::Int64)
# 	R = aⱼ[1] * ωⱼ[1] * cos(2π * zⱼ[1] * (τ + τⱼ[1])) * sinc(ωⱼ[1] * (τ + τⱼ[1]))
# 	for j in 2:J
# 		R += 2aⱼ[j] * ωⱼ[j] * cos(2π * zⱼ[j] * (τ + τⱼ[j])) * sinc(ωⱼ[j] * (τ + τⱼ[j]))
# 	end
# 	return R
# end

approximate_cross_spectral_density! (generic function with 1 method)

In [6]:
function BlockMatrix_from_cs(cs::CrossSpectralDensity, t₁::Vector{Float64}, t₂::Vector{Float64}, σ_X₁²::Vector{Float64}, σ_X₂²::Vector{Float64}, f0::Float64, fM::Float64, J::Int64)

	ωⱼ = Vector{Float64}(undef, J)
	zⱼ = Vector{Float64}(undef, J)

	a_𝓟₁ = Vector{Float64}(undef, J)
	a_𝓟₂ = Vector{Float64}(undef, J)
	a_𝓒₁₂ = Vector{Float64}(undef, J)
	a_τ = Vector{Float64}(undef, J)

	approximate_cross_spectral_density!(ωⱼ, zⱼ, a_𝓟₁, a_𝓟₂, a_𝓒₁₂, a_τ, cs, f0, fM, J)

	Σ₁₁ = Matrix{Float64}(undef, length(t₁), length(t₁))
	Σ₂₂ = Matrix{Float64}(undef, length(t₂), length(t₂))
	Σ₂₁ = Matrix{Float64}(undef, length(t₂), length(t₁))

	for (i, t₁ᵢ) in enumerate(t₁)
		for (j, t₁ⱼ) in enumerate(t₁)
			Σ₁₁[i, j] = approximated_covariance(t₁ᵢ - t₁ⱼ, a_𝓟₁, ωⱼ, zⱼ, J)
			if i == j
				Σ₁₁[i, j] += σ_X₁²[i]
			end
		end
	end

	for (i, t₂ᵢ) in enumerate(t₂)
		for (j, t₂ⱼ) in enumerate(t₂)
			Σ₂₂[i, j] = approximated_covariance(t₂ᵢ - t₂ⱼ, a_𝓟₂, ωⱼ, zⱼ, J)
			if i == j
				Σ₂₂[i, j] += σ_X₂²[i]
			end
		end
		for (j, t₁ⱼ) in enumerate(t₁)
			Σ₂₁[i, j] = approximated_cross_covariance(t₂ᵢ - t₁ⱼ, a_𝓒₁₂, a_τ, ωⱼ, zⱼ, J)
		end
	end
	return BlockCovarianceMatrix(Symmetric(Σ₁₁), Σ₂₁, Symmetric(Σ₂₂))
end

BlockMatrix_from_cs_2 (generic function with 1 method)

In [8]:
Σ = BlockMatrix_from_cs(cs, t,t, σ_x₁, σ_x₂, f0, fM, J)
Σ₁₁, Σ₂₁, Σ₂₂ = Σ.Σ₁₁, Σ.Σ₂₁, Σ.Σ₂₂

([0.17911595983604425 0.1159581957570659 … 0.0010335621265043034 0.0010283002356108156; 0.1159581957570659 0.12588153111584088 … 0.001054899757055121 0.0010335621265043034; … ; 0.0010335621265043034 0.001054899757055121 … 0.121573293264354 0.1159581957570659; 0.0010283002356108156 0.0010335621265043034 … 0.1159581957570659 0.12099798263719692], [0.10079062681121077 0.11768475560954224 … 0.0030826486690696546 0.003060593414510549; 0.08631097779534654 0.10079062681121077 … 0.003083550301481846 0.0030826486690696546; … ; 0.002164422765559585 0.0023303892803396837 … 0.10079062681121077 0.11768475560954224; 0.002078031251877148 0.002164422765559585 … 0.08631097779534654 0.10079062681121077], [0.37187866401821246 0.3051347583273216 … 0.005725148990888024 0.005753945500846771; 0.3051347583273216 0.4309390112889786 … 0.00580798750208375 0.005725148990888024; … ; 0.005725148990888024 0.00580798750208375 … 0.37918896033703725 0.3051347583273216; 0.005753945500846771 0.005725148990888024 … 0.3051

In [10]:
Σ = BlockMatrix_from_cs(cs, t,t, σ_x₁, σ_x₂, f0, fM, J)
Σ₁₁, Σ₂₁, Σ₂₂ = Σ.Σ₁₁, Σ.Σ₂₁, Σ.Σ₂₂
sanity_checks(Σ)
Σ₂ = BlockMatrix_from_cs_2(cs, t, t, σ_x₁, σ_x₂, f0, fM, J)
Σ₁₁₂, Σ₂₁₂, Σ₂₂₂ = Σ₂.Σ₁₁, Σ₂.Σ₂₁, Σ₂.Σ₂₂
@assert Σ₁₁ ≈ Σ₁₁₂ && Σ₂₁ ≈ Σ₂₁₂ && Σ₂₂ ≈ Σ₂₂₂ "Error 2"
sanity_checks(Σ₂)

In [11]:
@benchmark BlockMatrix_from_cs_3(cs, t, t, σ_x₁, σ_x₂, f0, fM, J)

BenchmarkTools.Trial: 119 samples with 1 evaluation.
 Range (min … max):  40.518 ms … 59.045 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     41.590 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   41.999 ms ±  1.932 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ██▄▃▁                                                   
  ▅▃▃█▅█████▆▆▅▃▁▄▃▄▃▃▁▁▃▃▁▃▁▃▁▁▁▁▃▁▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  40.5 ms         Histogram: frequency by time        48.9 ms <

 Memory estimate: 246.72 KiB, allocs estimate: 45.

In [12]:
@benchmark BlockMatrix_from_cs_2(cs, t, t, σ_x₁, σ_x₂, f0, fM, J)

BenchmarkTools.Trial: 120 samples with 1 evaluation.
 Range (min … max):  40.971 ms …  45.248 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     41.786 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   41.883 ms ± 624.170 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▂   █▂  ▁ ▁▄  ▁▁                                         
  ▃▃▃▅▅███████▆█▃███▆███▆▃▆▅▅█▃▅▁▃▁█▅▁▃▃▁▃▁▁▁▁▃▁▁▃▁▃▁▁▁▁▁▁▁▁▁▃ ▃
  41 ms           Histogram: frequency by time           44 ms <

 Memory estimate: 246.69 KiB, allocs estimate: 43.

In [13]:
@benchmark BlockMatrix_from_cs(cs, t, t, σ_x₁, σ_x₂, f0, fM, J)  

BenchmarkTools.Trial: 117 samples with 1 evaluation.
 Range (min … max):  39.112 ms … 67.295 ms  ┊ GC (min … max): 1.75% … 1.23%
 Time  (median):     42.032 ms              ┊ GC (median):    1.75%
 Time  (mean ± σ):   42.857 ms ±  4.506 ms  ┊ GC (mean ± σ):  1.93% ± 1.11%

  ▁▂ ▄ ▇ █▄▁                                                   
  ██▆███▆███▇▄▇▃▄▆▁▃▁▃▁▃▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▄ ▃
  39.1 ms         Histogram: frequency by time        62.6 ms <

 Memory estimate: 46.12 MiB, allocs estimate: 1248.

In [ ]:
# function get_chi2term(L₁₁::LowerTriangular{Float64,Array{Float64,2}}, L₂₁::Array{Float64,2}, L₂₂::LowerTriangular{Float64,Array{Float64,2}}, Σ₂₁::Array{Float64,2}, x₁::Vector{Float64}, x₂::Vector{Float64})
# 	z₁ = L₁₁ \ x₁
# 	w = L₂₂ \ x₂
# 	v = L₁₁' \ z₁
# 	g = Σ₂₁ * v
# 	u = L₂₂ \ g
# 	z₂ = w - u
# 	# return z₁, z₂
# 	return z₁' * z₁ + z₂' * z₂
# end
# function log_likelihood(cs::CrossSpectralDensity, t₁::Vector{Float64}, t₂::Vector{Float64}, x₁::Vector{Float64}, x₂::Vector{Float64}, σ_X₁²::Vector{Float64}, σ_X₂²::Vector{Float64}, f0::Float64, fM::Float64, J::Int)

# 	Σ = BlockMatrix_from_cs(cs, t₁, t₂, σ_X₁², σ_X₂², f0, fM, J)
# 	L₁₁, L₂₁, L₂₂, schur = cholesky(Σ)

# 	return -0.5 * get_chi2term(L₁₁, L₂₁, L₂₂, Σ₂₁, x₁, x₂) - 0.5 * (logdet(Σ₁₁) + logdet(schur))
# end
log_likelihood(cs, t, t, x₁, x₂, σ_x₁.^2, σ_x₂.^2, f0, fM, J)

In [ ]:
@benchmark log_likelihood(cs, t, t, x₁, x₂, σ_x₁, σ_x₂, f0, fM, J)

In [9]:
using ProfileView

In [ ]:
ProfileView.@profview log_likelihood(cs, t, t, x₁, x₂, σ_x₁, σ_x₂, f0, fM, J)

In [6]:
using BenchmarkTools

In [ ]:
@benchmark log_likelihood(cs, t, t, x₁, x₂, σ_x₁, σ_x₂, f0, fM, J)